In [1]:
import os
import sarge
from multiprocessing import Pool

In [2]:
os.chdir('../../../Rat-Pilot/data/tag_directories/')

Peak finding

In [3]:
tag_dirs = [('H3K27ac1-IL/','Input1-IL/'),('H3K27ac2-PL/','Input2-PL/'),('H3K27ac4-ST/','Input4-ST/'),('H3K27ac5-AC/','Input5-AC/'),
            ('H3K27ac6-AS/','Input6-AS/'),('H3K27ac7-VTA/','Input7-VTA/')]

In [4]:
'''
Define a function that can perform peak finding when given a target tag directory and an input tag directory.
'''
def find_peaks(args):
    
    target_dir,input_dir = args
    
    cmd = f'findPeaks {target_dir} -i {input_dir} -style histone -o auto'
    
    print(cmd)
    sarge.run(cmd)

In [5]:
pool = Pool(processes=len(tag_dirs))
pool.map(find_peaks,tag_dirs)
pool.close()

findPeaks H3K27ac1-IL/ -i Input1-IL/ -style histone -o auto
findPeaks H3K27ac2-PL/ -i Input2-PL/ -style histone -o auto
findPeaks H3K27ac4-ST/ -i Input4-ST/ -style histone -o auto
findPeaks H3K27ac7-VTA/ -i Input7-VTA/ -style histone -o auto
findPeaks H3K27ac5-AC/ -i Input5-AC/ -style histone -o auto
findPeaks H3K27ac6-AS/ -i Input6-AS/ -style histone -o auto


Merge peaks

In [6]:
!mergePeaks */regions.txt > all_peaks.txt

	Max distance to merge: direct overlap required (-d given)
	Merging peaks... 
	Comparing H3K27ac1-IL/regions.txt (79380 total) and H3K27ac1-IL/regions.txt (79380 total)
	Comparing H3K27ac1-IL/regions.txt (79380 total) and H3K27ac2-PL/regions.txt (100039 total)
	Comparing H3K27ac1-IL/regions.txt (79380 total) and H3K27ac4-ST/regions.txt (74575 total)
	Comparing H3K27ac1-IL/regions.txt (79380 total) and H3K27ac5-AC/regions.txt (72906 total)
	Comparing H3K27ac1-IL/regions.txt (79380 total) and H3K27ac6-AS/regions.txt (69392 total)
	Comparing H3K27ac1-IL/regions.txt (79380 total) and H3K27ac7-VTA/regions.txt (66509 total)
	Comparing H3K27ac2-PL/regions.txt (100039 total) and H3K27ac1-IL/regions.txt (79380 total)
	Comparing H3K27ac2-PL/regions.txt (100039 total) and H3K27ac2-PL/regions.txt (100039 total)
	Comparing H3K27ac2-PL/regions.txt (100039 total) and H3K27ac4-ST/regions.txt (74575 total)
	Comparing H3K27ac2-PL/regions.txt (100039 total) and H3K27ac5-AC/regions.txt (72906 total)
	Comp

In [7]:
!pos2bed.pl all_peaks.txt > all_peaks.bed


	Converted 225074 peaks total



Annotate Peaks and Get Expression

In [8]:
!annotatePeaks.pl all_peaks.txt rn6 -raw -cpu 6 -d H3K27ac* > all_peaks.raw.txt


	Peak file = all_peaks.txt
	Genome = rn6
	Organism = rat
	Will NOT normalize tag counts
	Will use up to 6 CPUs in parts that can use them
	Tag Directories:
		H3K27ac1-IL
		H3K27ac2-PL
		H3K27ac4-ST
		H3K27ac5-AC
		H3K27ac6-AS
		H3K27ac7-VTA
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 225074
		Duplicated Peak IDs: 0

	Peak File Statistics:
		Total Peaks: 225074
		Redundant Peak IDs: 0
		Peaks lacking information: 0 (need at least 5 columns per peak)
		Peaks with misformatted coordinates: 0 (should be integer)
		Peaks with misformatted strand: 0 (should be either +/- or 0/1)

	Peak file looks good!

	Reading Positions...
	-----------------------
	Finding Closest TSS...
	Annotating:.........................
		Annotation	Number of peaks	Total size (bp)	Log2 Enrichment
		3UTR	1617.0	9155204	1.124
		ncRNA	28.0	179999	0.941
		TTS	2592.0	18580415	0.783
		pseudo	1.0	39928	-1.694
		Exon	5698.0	24903938	1.497
		Intron	98550.0	702186994	0.792
		I

Create a bigwig hub

In [9]:
!makeMultiWigHub.pl rat-pilot rn6 -url http://homer.ucsd.edu/sjroth/Rat-Pilot -webdir /homer_data/www/html/sjroth/Rat-Pilot -d H3K27ac* Input*

Colors that will be used (change with -color or -gradient options):
	Index	Color	Neg. color	Tag Directory
	1	255,150,150	(255,180,180)	H3K27ac1-IL
	2	150,150,255	(180,180,255)	H3K27ac2-PL
	3	150,255,150	(180,255,180)	H3K27ac4-ST
	4	255,200,150	(255,210,180)	H3K27ac5-AC
	5	150,255,220	(180,255,240)	H3K27ac6-AS
	6	200,150,255	(210,180,255)	H3K27ac7-VTA
	7	200,200,150	(210,210,170)	Input1-IL
	8	150,200,200	(170,210,210)	Input2-PL
	9	200,150,200	(210,170,210)	Input4-ST
	10	194,83,59	(219,78,81)	Input5-AC
	11	140,73,134	(236,194,139)	Input6-AS
	12	110,170,90	(74,50,103)	Input7-VTA


	Once finished, you will want to upload the following hub URL:
		http://homer.ucsd.edu/sjroth/Rat-Pilot/rat-pilot/hub.txt

	If loading to the Wash U Epigenome Browser, use:
		http://homer.ucsd.edu/sjroth/Rat-Pilot/rat-pilot/washU.hub.txt


	Visualization fragment length = 177
	Output file: H3K27ac1-IL/H3K27ac1-IL.ucsc.bigWig
	No need to remove tags to get desired file size
	Generating bedGraph for chr1
	Generati